In [15]:
import json

In [17]:
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

In [18]:
prompt_label_remap = {
    "q7": "Inv. Scaling",
    "q9": "BigBench CoT",
}

In [19]:
all_printed_outs = {}

def arrow(diff):
    if diff > 0:
        return f"dab{{{str(round(diff * 100, 1))}}}"
    else:
        return f"uag{{{str(-1 * round(diff * 100, 1))}}}"

def print_table(curr_data, data):
    labels = []
    men_means = []
    women_means = []
    pm_men = []
    pm_women = []
    for k in curr_data:
        # cond = 11 > int(k['prompt'][1:]) > 5        
        if k["data"] == data:
            prompt_label = prompt_label_remap[f"{k['prompt']}"]
            if (len(labels) > 0 and labels[-1] != prompt_label) or len(labels) == 0:
                labels.append(prompt_label)
            if k["cot"] == True:
                women_means.append(k["bounds"][1])
                pm_women.append(k["bounds"][1] - k["bounds"][0])
            else:
                men_means.append(k["bounds"][1])
                pm_men.append(k["bounds"][1] - k["bounds"][0])

    print(labels)

    for idx, label in enumerate(labels):
        diff = men_means[idx] - women_means[idx]
        row = f" & \\texttt{{{label}}} & ${str(round(men_means[idx] * 100))} \pm {str(round(pm_men[idx] * 100))} \%$ & \\{arrow(diff)} ${str(round(women_means[idx] * 100))} \pm {str(round(pm_women[idx] * 100))} \\%$ "
        if (data, label) in all_printed_outs:
            all_printed_outs[(data, label)] += row[len(f" & \\texttt{{{label}}} "):] + " & "
        else: all_printed_outs[(data, label)] = row


In [20]:
all_printed_outs = {}


with open("./stats.json") as f:
    d = json.load(f)
    # for model in ["text-davinci-001", "text-davinci-002", "text-davinci-003"]:
    for model in ["text-davinci-003"]:
        curr_out = [x for x in d if x["model"] == model]
        curr_out = sorted(curr_out, key=lambda x: x["prompt"])
        for k in ["crows", "stereoset", "bbq"]:
            try:
                print_table(curr_out, k)
            except:
                continue

for k in all_printed_outs:
    print(all_printed_outs[k])

['Inv. Scaling', 'BigBench CoT']
['Inv. Scaling', 'BigBench CoT']
['Inv. Scaling', 'BigBench CoT']
 & \texttt{Inv. Scaling} & $60 \pm 0 \%$ & \uag{2.1} $62 \pm 1 \%$ 
 & \texttt{BigBench CoT} & $73 \pm 1 \%$ & \uag{4.3} $77 \pm 1 \%$ 
 & \texttt{Inv. Scaling} & $49 \pm 0 \%$ & \dab{9.3} $40 \pm 1 \%$ 
 & \texttt{BigBench CoT} & $55 \pm 1 \%$ & \dab{2.4} $52 \pm 1 \%$ 
 & \texttt{Inv. Scaling} & $89 \pm 0 \%$ & \uag{0.1} $89 \pm 1 \%$ 
 & \texttt{BigBench CoT} & $71 \pm 0 \%$ & \uag{17.7} $88 \pm 1 \%$ 
